In [13]:
import os
import json

config_file = os.path.join(os.path.abspath(''), os.pardir, 'scripts', 'config.json')
with open(config_file) as json_data:
    config = json.load(json_data)

fmriprep_dir = config['fmriprep_dir']